In [1]:
import torch
import argparse
import time
import math
import os, sys
import itertools
import pickle
from tqdm import tqdm
import numpy as np
import os.path as osp

In [2]:
torch.cuda.device_count()

1

In [3]:
def loadpkl(path):
    with open(path, 'rb') as f:
        rst=pickle.load(f)
    return rst

In [4]:
path="/share/data/mei-work/kangrui/github/ssref/data/refsum-data/arxiv-aiml/full_id_abs_title_cite_ref_author.pkl"
full_data=loadpkl(path)

In [5]:
path="/share/data/mei-work/kangrui/github/ssref/result/eval_rerank/test_result/eval_key_id2embed.pkl"
key_embedding=loadpkl(path)

In [6]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_pred_1000.pkl"
pred_set=loadpkl(path)

In [7]:
path="/share/data/mei-work/kangrui/github/ssref/result/pretrained_pair_sbert/f1_result/eval_test_gold.pkl"
gold_set=loadpkl(path)

In [8]:
query_id2embedding=key_embedding
key_id2embedding=key_embedding

In [9]:
candidate_set={}
candidate_ref_set={}
for k,v in pred_set.items():
    cur=set()
    for paper in v[:200]:
        cur.add(paper)
        refset=set()
        for ref in full_data[paper]["references"]:
            cur.add(ref)
            refset.add(ref)
        candidate_ref_set[paper]=refset
    candidate_set[k]=list(cur)

In [10]:
def rerank_weighted_sum(full_data,candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,rerank_num=None,device=None):
    cnt=0
    pbar = tqdm(range(0,len(candidate_set),1), postfix=f"calculating f1")
    query=list(candidate_set.keys())
    empty_embed=torch.tensor(np.zeros_like(next(iter(key_id2embedding.values()))))
    empty_embed_cnt=0
    if rerank_num is None:
        for v in candidate_set.values():
            if rerank_num is None or rerank_num>len(v):
                rerank_num=len(v)
                
    rerank_candidate={}   
    for i in pbar:
        query_paper=query[i]
        query_embedding=torch.tensor(query_id2embedding[query_paper]).unsqueeze(0)
        
        cited_dict={}
        for candidate in candidate_set[query_paper]:
            
            if candidate in candidate_ref_set.keys():
                # this is possibly the anchor paper
                for ref in list(candidate_ref_set[candidate]):
                    # deal with the ref of an anchor paper
                    if ref in candidate_set[query_paper]:
                        if ref not in cited_dict.keys():
                            cited_dict[ref]=[candidate]
                        else:
                            cited_dict[ref].append(candidate)
                    else:
                        # this is not anchor paper for this query
                        break
            if candidate not in cited_dict.keys():
                cited_dict[candidate]=[candidate]
            else:
                cited_dict[candidate].append(candidate)
                
                
        key_embedding=[]
        for candidate in candidate_set[query_paper]:
            candidate_embedding=[]
            total_weight=1
            for paper in cited_dict[candidate]:
                if paper in key_id2embedding:
                    weight=full_data[paper]['citationCount']
                    total_weight+= weight
                    candidate_embedding.append(weight*torch.tensor(key_id2embedding[paper]))
            if len(candidate_embedding)==0:
                candidate_embedding.append(empty_embed)
                empty_embed_cnt+=1
            candidate_embedding=torch.sum(torch.stack(candidate_embedding),dim=0)/total_weight
            
            key_embedding.append(candidate_embedding)
        
        
        key_embedding=torch.stack(key_embedding)
        query_embedding=query_embedding.to(device)
        key_embedding=key_embedding.to(device)
        pred_logits=torch.mm(query_embedding,key_embedding.T)
        top_k=torch.topk(pred_logits[0],k=rerank_num)[1]
#         print(pred_logits[0][top_k[-5:]])
        rerank=[candidate_set[query_paper][idx.cpu().item()] for idx in top_k]
        rerank_candidate[query_paper]=rerank
    print("empty embedding cnt->",empty_embed_cnt)
    print("min rerank num->",rerank_num)
    return rerank_candidate


        


In [11]:
def rerank_weighted(full_data,candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,rerank_num=None,device=None):
    cnt=0
    pbar = tqdm(range(0,len(candidate_set),1), postfix=f"calculating f1")
    query=list(candidate_set.keys())
    empty_embed=torch.tensor(np.zeros_like(next(iter(key_id2embedding.values()))))
    empty_embed_cnt=0
    if rerank_num is None:
        for v in candidate_set.values():
            if rerank_num is None or rerank_num>len(v):
                rerank_num=len(v)
                
    rerank_candidate={}   
    for i in pbar:
        query_paper=query[i]
        query_embedding=torch.tensor(query_id2embedding[query_paper]).unsqueeze(0)
        
        cited_dict={}
        for candidate in candidate_set[query_paper]:
            
            if candidate in candidate_ref_set.keys():
                # this is possibly the anchor paper
                for ref in list(candidate_ref_set[candidate]):
                    # deal with the ref of an anchor paper
                    if ref in candidate_set[query_paper]:
                        if ref not in cited_dict.keys():
                            cited_dict[ref]=[candidate]
                        else:
                            cited_dict[ref].append(candidate)
                    else:
                        # this is not anchor paper for this query
                        break
            if candidate not in cited_dict.keys():
                cited_dict[candidate]=[candidate]
            else:
                cited_dict[candidate].append(candidate)
                
                
        key_embedding=[]
        for candidate in candidate_set[query_paper]:
            candidate_embedding=[]
            total_weight=1
            for paper in cited_dict[candidate]:
                if paper in key_id2embedding:
                    weight=full_data[paper]['citationCount']
                    total_weight+= weight
                    candidate_embedding.append(weight*torch.tensor(key_id2embedding[paper]))
            if len(candidate_embedding)==0:
                candidate_embedding.append(empty_embed)
                empty_embed_cnt+=1
            candidate_embedding=torch.sum(torch.stack(candidate_embedding),dim=0)
            
            key_embedding.append(candidate_embedding)
        
        
        key_embedding=torch.stack(key_embedding)
        query_embedding=query_embedding.to(device)
        key_embedding=key_embedding.to(device)
        pred_logits=torch.mm(query_embedding,key_embedding.T)
        top_k=torch.topk(pred_logits[0],k=rerank_num)[1]
#         print(pred_logits[0][top_k[-5:]])
        rerank=[candidate_set[query_paper][idx.cpu().item()] for idx in top_k]
        rerank_candidate[query_paper]=rerank
    print("empty embedding cnt->",empty_embed_cnt)
    print("min rerank num->",rerank_num)
    return rerank_candidate



In [12]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [13]:
rerank_candidate=rerank_weighted_sum(full_data,candidate_set,query_id2embedding,key_id2embedding,candidate_ref_set,device=device)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 944/944 [06:59<00:00,  2.25it/s, calculating f1]

empty embedding cnt-> 0
min rerank num-> 1891


In [14]:
sys.path.append("/share/data/mei-work/kangrui/github/ssref")
from utils.test_utils import *
from utils.utils import *
def show_rerank_rst(rerank_candidate,gold_set):
    pool_length=len(next(iter(rerank_candidate.values())))
    i=2
    while 1:
        all_pred_topN=[]
        all_gold=[]
        for query_paper,paperlist in rerank_candidate.items():
            topN=paperlist[:i]
            all_pred_topN.append(topN)
            all_gold.append(gold_set[query_paper])
        all_pred_topN=np.array(all_pred_topN)
        metric = get_precision_recall_f1(all_pred_topN, all_gold)
        print(f"top{i}:\n{metric}")
        if i>pool_length:
            break
        i*=2

In [15]:
show_rerank_rst(rerank_candidate,gold_set)

top2:
{'precision': 0.10434322033898305, 'recall': 0.005124869927159209, 'f1': 0.009769886927196984, 'true_pos': 197, 'false_pos': 1691, 'false_neg': 38243, 'num_sample': 944}
top4:
{'precision': 0.0847457627118644, 'recall': 0.008324661810613945, 'f1': 0.01516012886109532, 'true_pos': 320, 'false_pos': 3456, 'false_neg': 38120, 'num_sample': 944}
top8:
{'precision': 0.07971398305084745, 'recall': 0.01566077003121748, 'f1': 0.026178465820142632, 'true_pos': 602, 'false_pos': 6950, 'false_neg': 37838, 'num_sample': 944}
top16:
{'precision': 0.0697166313559322, 'recall': 0.02739334027055151, 'f1': 0.039332138054683996, 'true_pos': 1053, 'false_pos': 14051, 'false_neg': 37387, 'num_sample': 944}
top32:
{'precision': 0.06197033898305085, 'recall': 0.04869927159209157, 'f1': 0.05453909800722526, 'true_pos': 1872, 'false_pos': 28336, 'false_neg': 36568, 'num_sample': 944}
top64:
{'precision': 0.05334679555084746, 'recall': 0.08384495317377731, 'f1': 0.06520595613822125, 'true_pos': 3223, 'fa